<a href="https://colab.research.google.com/github/FrancescaGraziaRadatti/Thesis/blob/main/v9_final_version_codes_for_Thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#1.x tensor flow version 

%tensorflow_version 1.x
import tensorflow as tf

print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [6]:
#import packages to install Spark
!pip install pyspark
import pyspark
from pyspark.sql import SparkSession
import seaborn as sns   
import matplotlib.pyplot as plt  
import pandas as pd
import numpy as np

     |████████████████████████████████| 281.3 MB 36 kB/s 
     |████████████████████████████████| 198 kB 50.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=b7c1baf1d6831289f4fab91d3da5ed389adf827bab49a9ec50e8b83a642bab58
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [7]:
from keras.models import Sequential,Model
from keras.layers import Dense,Dropout,Activation
from keras.layers import Flatten,Input
from keras.layers import Embedding
from keras.layers import concatenate
from tensorflow.keras.utils import to_categorical
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import Bidirectional
from keras.layers import Dense, Embedding, LSTM, GRU, Flatten, Dropout, Lambda
from keras.layers.embeddings import Embedding
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

Using TensorFlow backend.


In [9]:
#import libraries 
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split

In [2]:
#load reviews from Drive 
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
!ls "/content/drive/My Drive/reviews_Clothing_Shoes_and_Jewelry.json.gz"

'/content/drive/My Drive/reviews_Clothing_Shoes_and_Jewelry.json.gz'


In [5]:
!ls "/content/drive/MyDrive/meta_Clothing_Shoes_and_Jewelry.json.gz"

/content/drive/MyDrive/meta_Clothing_Shoes_and_Jewelry.json.gz


In [8]:
#build the spark function 
spark = SparkSession.builder.enableHiveSupport().getOrCreate()

In [10]:
#read the json file with spark 
df_reviews = spark.read.json("/content/drive/MyDrive/reviews_Clothing_Shoes_and_Jewelry.json.gz")


In [11]:
#brief overview of just created reviews dataframe
df_reviews.show(n=25)

+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|      asin| helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|0000031887|  [0, 0]|    5.0|Perfect red tutu ...| 11 4, 2013|A2XVJBSRI3SWDI|             abigail|           Nice tutu|    1383523200|
|0000031887|  [1, 1]|    4.0|This was a really...|05 26, 2012|A2G0LNLN79Q6HR|       aj_18 "Aj_18"|Really Cute but r...|    1337990400|
|0000031887|  [1, 1]|    2.0|the tutu color wa...|02 17, 2013|A2R3K1KX09QBYP|      alert consumer|not very good mat...|    1361059200|
|0000031887|  [0, 1]|    1.0|My 3-yr-old daugh...|03 21, 2013| A19PBP93OF896|Alinna Satake "Ca...|Tiny and Poorly C...|    1363824000|
|0000031887|  [0, 0]|    4.0|Bought it for my ...|01 23

In [12]:
#read the products dataframe
df_products = spark.read.json("/content/drive/MyDrive/meta_Clothing_Shoes_and_Jewelry.json.gz")
df_products.show(n=25)

+--------------------+----------+--------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+
|     _corrupt_record|      asin|         brand|          categories|         description|               imUrl|price|             related|           salesRank|               title|
+--------------------+----------+--------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+
|                null|0000037214|    Big Dreams|[[Clothing, Shoes...|                null|http://ecx.images...| 6.99|{null, [B00JO8II7...|{null, null, null...|Purple Sequin Tin...|
|                null|0000031887|Boutique Cutie|[[Clothing, Shoes...|This adorable bas...|http://ecx.images...| 6.79|{[0000031852, 000...|{null, null, null...|Ballet Dress-Up F...|
|                null|0123456479|          null|[[Clothing, Shoes...|Elegance par exce...|http:

Some of the columns inside both dataFrames need to be deleted (since there are not significant). Moreover, some columns that are expressed via array need to be flattened applying the transform and flatten functions. 


For df_products, the arrays are: also_buy, also_view, category, description, feature, and image. Details variable is a structure. 

For df_reviews, style is a structure, image is an array e verified is a boolean. 

In [ ]:
df_products.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- description: string (nullable = true)
 |-- imUrl: string (nullable = true)
 |-- price: double (nullable = true)
 |-- related: struct (nullable = true)
 |    |-- also_bought: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- also_viewed: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- bought_together: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- buy_after_viewing: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- salesRank: struct (nullable = true)
 |    |-- Arts, Crafts & Sewing: long (nullable = true)
 |    |-- Automotive: long (nullable = true)
 |    |-- Baby: long (null

In [ ]:
df_reviews.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



In [13]:
def spark_shape(self):
    return (self.count(), len(self.columns))
pyspark.sql.dataframe.DataFrame.shape = spark_shape


In [ ]:
#shape
spark_shape(df_products)
#shape
spark_shape(df_reviews)

(5748920, 9)

The shapes are different and the ASIN variable is my join key. In the following codes I'll try to remove null values using some logics in order to have a clean marged dataFrame at the end 

**Data preprocessing**

- remove nullvalues (delete or fill with other values when possible)

- remove variables not useful to aim

-merge dataFrame and drop duplicated values for the primary key (asin, ReviewerName and unixReviewTime)

-Analyse again the merged dataFrame

In [ ]:
from pyspark.sql.functions import isnan, when, count, col
df_products.select([count(when(col(c).isNull(), c)).alias(c) for c in df_products.columns]).show()

+---------------+-----+-------+----------+-----------+-----+------+-------+---------+-----+
|_corrupt_record| asin|  brand|categories|description|imUrl| price|related|salesRank|title|
+---------------+-----+-------+----------+-----------+-----+------+-------+---------+-----+
|        1488673|14711|1409043|     14711|    1432653|14790|936998| 456742|    94634|15315|
+---------------+-----+-------+----------+-----------+-----+------+-------+---------+-----+



In [14]:
#drop null values from 'reviewerName','reviewText','summary','title'; I can't delete the 'description' variable since it has too many null vlaues 
df_prd_cleaned= df_products.na.drop(subset=["title"])

In [ ]:
df_prd_cleaned.select([count(when(col(c).isNull(), c)).alias(c) for c in df_prd_cleaned.columns]).show()

+---------------+----+-------+----------+-----------+-----+------+-------+---------+-----+
|_corrupt_record|asin|  brand|categories|description|imUrl| price|related|salesRank|title|
+---------------+----+-------+----------+-----------+-----+------+-------+---------+-----+
|        1488069|   0|1393728|         0|    1417827|    0|922095| 441895|    79853|    0|
+---------------+----+-------+----------+-----------+-----+------+-------+---------+-----+



In [17]:
from pyspark.sql.functions import *

df_prd_cleaned1 = df_prd_cleaned.withColumn('brand_title', pyspark.sql.functions.split(df_prd_cleaned['title'],' ').getItem(0)) 
df_prd_cleaned1.show(n=20)

+---------------+----------+--------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+-----------+
|_corrupt_record|      asin|         brand|          categories|         description|               imUrl|price|             related|           salesRank|               title|brand_title|
+---------------+----------+--------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+-----------+
|           null|0000037214|    Big Dreams|[[Clothing, Shoes...|                null|http://ecx.images...| 6.99|{null, [B00JO8II7...|{null, null, null...|Purple Sequin Tin...|     Purple|
|           null|0000031887|Boutique Cutie|[[Clothing, Shoes...|This adorable bas...|http://ecx.images...| 6.79|{[0000031852, 000...|{null, null, null...|Ballet Dress-Up F...|     Ballet|
|           null|0123456479|          null|[[Clothing, Shoes

In [ ]:
spark_shape(df_prd_cleaned1)

(1488069, 11)

It could be not possible to take brand values from title values; it is better delete values since I want to analyse also brand differences among products and sentiment polarity for brands, so I want to have clean value for brand 

In [ ]:
from pyspark.sql.functions import when
df_prd_cleaned2= df_prd_cleaned1.select('brand_title',
           when(df_prd_cleaned1.brand.isNull(), df_prd_cleaned1.brand_title).otherwise(df_prd_cleaned1.brand)).show()

+-----------+---------------------------------------------------------+
|brand_title|CASE WHEN (brand IS NULL) THEN brand_title ELSE brand END|
+-----------+---------------------------------------------------------+
|     Purple|                                               Big Dreams|
|     Ballet|                                           Boutique Cutie|
|    SHINING|                                                  SHINING|
|        RiZ|                                                      RiZ|
|     Lantin|                                                   Lantin|
|        NVC|                                                      NVC|
|    Kismeth|                                                  Kismeth|
|    Max-MPH|                                                  Max-MPH|
|        FX1|                                                      FX1|
|        Riz|                                                      Riz|
|        Red|                                                   

Brand is different from brand_title for values for which brand is defined, so I have to delete brand null values

In [ ]:
df_prd_cleaned1.createOrReplaceTempView("df_prd_cleaned1")

query1 = """SELECT COUNT(*)
           FROM df_prd_cleaned2
           WHERE brand = brand_title
           """


In [ ]:
df_prd_cleaned1.createOrReplaceTempView("df_prd_cleaned2")

brand_title_similarity = spark.sql(query1)
brand_title_similarity.show()

+--------+
|count(1)|
+--------+
|   30731|
+--------+



In [ ]:
query2 = """SELECT COUNT(*)
           FROM df_prd_cleaned2
           WHERE brand <> brand_title
           """
brand_title_dissimilarity = spark.sql(query2)
brand_title_dissimilarity.show()

+--------+
|count(1)|
+--------+
|   63610|
+--------+



In [ ]:
query3 = """SELECT COUNT(*)
           FROM df_prd_cleaned2
           WHERE description is NULL 
           """
null_descr = spark.sql(query3)
null_descr.show()

+--------+
|count(1)|
+--------+
| 1417827|
+--------+



In [ ]:
type(df_prd_cleaned2)
type(df_prd_cleaned1)

pyspark.sql.dataframe.DataFrame


Brand cannot substitute by the first part of the title, since there are more different strings than similar. I decide to delete all observations having null values for brand. 

In [ ]:
df_prd_cleaned1.createOrReplaceTempView("df_prd_cleaned1")

query4 = """SELECT COUNT(*)
           FROM df_prd_cleaned1
           WHERE brand is NULL
           """
null_brand = spark.sql(query4)
null_brand.show()

+--------+
|count(1)|
+--------+
| 1393728|
+--------+



In [ ]:
df_prd_cleaned.select([count(when(col(c).isNull(), c)).alias(c) for c in df_prd_cleaned.columns]).show()

+--------+
|count(1)|
+--------+
| 1393728|
+--------+



In [ ]:
#description column cannot be used 
df_prd_cleaned1= df_prd_cleaned1.drop('description', 'brand_title')
spark_shape(df_prd_cleaned1)

(1488069, 9)

In [33]:
df_prd_cleaned3= df_prd_cleaned1.na.drop(subset=["brand"])

In [ ]:
df_prd_cleaned3.select([count(when(col(c).isNull(), c)).alias(c) for c in df_prd_cleaned3.columns]).show()

+---------------+----+-----+----------+-----+-----+-------+---------+-----+
|_corrupt_record|asin|brand|categories|imUrl|price|related|salesRank|title|
+---------------+----+-----+----------+-----+-----+-------+---------+-----+
|          94341|   0|    0|         0|    0|  675|   4994|    15376|    0|
+---------------+----+-----+----------+-----+-----+-------+---------+-----+



In [ ]:
query5 = """SELECT avg(price)
           FROM df_prd_cleaned1
           WHERE PRICE is NOT NULL 
           """
avg_price = spark.sql(query5)
avg_price.show()

+-----------------+
|       avg(price)|
+-----------------+
|46.55125714224298|
+-----------------+



In [34]:
##Replace the average price for null values in price column
df_prd_cleaned4= df_prd_cleaned3.fillna(46.55,subset=["price"])
df_prd_cleaned4.show()
spark_shape(df_prd_cleaned4)

+---------------+----------+-------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+
|_corrupt_record|      asin|              brand|          categories|               imUrl|price|             related|           salesRank|               title|
+---------------+----------+-------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+
|           null|0000037214|         Big Dreams|[[Clothing, Shoes...|http://ecx.images...| 6.99|{null, [B00JO8II7...|{null, null, null...|Purple Sequin Tin...|
|           null|0000031887|     Boutique Cutie|[[Clothing, Shoes...|http://ecx.images...| 6.79|{[0000031852, 000...|{null, null, null...|Ballet Dress-Up F...|
|           null|0681256036|               NECA|[[Clothing, Shoes...|http://ecx.images...|17.59|{null, [B005TLMG0...|{null, null, null...|Twilight Saga: Ne...|
|           null|1848572972| Little Tige

(94341, 9)

In [ ]:
df_prd_cleaned4.select([count(when(col(c).isNull(), c)).alias(c) for c in df_prd_cleaned2.columns]).show()

+---------------+----+-----+----------+-----+-----+-------+---------+-----+
|_corrupt_record|asin|brand|categories|imUrl|price|related|salesRank|title|
+---------------+----+-----+----------+-----+-----+-------+---------+-----+
|          94341|   0|    0|         0|    0|    0|   4994|    15376|    0|
+---------------+----+-----+----------+-----+-----+-------+---------+-----+



In [ ]:
#createTemporaryView for df_prd2
#df_prd_cleaned2.createOrReplaceTempView("df_prd_cleaned2")


Now I have to take only title, price, asin and category variables. So i have to delete all the other variables and then I have to merge this dataset to the review dataset and finally i have to remove duplicates for the primary key. 

-after marged, I can compute n' of reviewers for product and also other descriptive analysis done by someone in the links saved by me. 


In [35]:
#delete not necessary columns 
df_prd_cleaned5= df_prd_cleaned4.drop('_corrupt_record','related', 'salesRank', 'imUrl')
type(df_prd_cleaned5)

pyspark.sql.dataframe.DataFrame

In [ ]:
df_prd_cleaned5.dtypes

[('asin', 'string'),
 ('brand', 'string'),
 ('categories', 'array<array<string>>'),
 ('price', 'double'),
 ('title', 'string')]

In [ ]:
spark_shape(df_prd_cleaned5)

(94341, 6)

In [ ]:
df_prd_fin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94341 entries, 0 to 94340
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   asin        94341 non-null  object 
 1   brand       94341 non-null  object 
 2   categories  94341 non-null  object 
 3   price       94341 non-null  float64
 4   title       94341 non-null  object 
dtypes: float64(1), object(4)
memory usage: 3.6+ MB


Analyse Review DataFrame now


In [85]:
#count null values for reviews 
df_reviews.select([count(when(col(c).isNull(), c)).alias(c) for c in df_reviews.columns]).show()

+----+-------+-------+----------+----------+----------+------------+-------+--------------+
|asin|helpful|overall|reviewText|reviewTime|reviewerID|reviewerName|summary|unixReviewTime|
+----+-------+-------+----------+----------+----------+------------+-------+--------------+
|   0|      0|      0|         0|         0|         0|       13698|      0|             0|
+----+-------+-------+----------+----------+----------+------------+-------+--------------+



In [24]:
#now I have to delete null values for reviewerName 
df_reviews_cleaned= df_reviews.na.drop(subset=["reviewerName"])

In [ ]:
df_reviews_cleaned.show(n=20)
spark_shape(df_reviews_cleaned)

+----------+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|      asin|helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+----------+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|0000031887| [0, 0]|    5.0|Perfect red tutu ...| 11 4, 2013|A2XVJBSRI3SWDI|             abigail|           Nice tutu|    1383523200|
|0000031887| [1, 1]|    4.0|This was a really...|05 26, 2012|A2G0LNLN79Q6HR|       aj_18 "Aj_18"|Really Cute but r...|    1337990400|
|0000031887| [1, 1]|    2.0|the tutu color wa...|02 17, 2013|A2R3K1KX09QBYP|      alert consumer|not very good mat...|    1361059200|
|0000031887| [0, 1]|    1.0|My 3-yr-old daugh...|03 21, 2013| A19PBP93OF896|Alinna Satake "Ca...|Tiny and Poorly C...|    1363824000|
|0000031887| [0, 0]|    4.0|Bought it for my ...|01 23, 2014|A

(5735222, 9)

In [ ]:
type(df_reviews_cleaned)

pyspark.sql.dataframe.DataFrame

In [ ]:
df_reviews_cleaned.dtypes

[('asin', 'string'),
 ('helpful', 'array<bigint>'),
 ('overall', 'double'),
 ('reviewText', 'string'),
 ('reviewTime', 'string'),
 ('reviewerID', 'string'),
 ('reviewerName', 'string'),
 ('summary', 'string'),
 ('unixReviewTime', 'bigint')]

In [25]:
#extract year from reviews: try to compute min and max of review year --- prima calcolare year, poi trasformarlo in integer e poi calcolare min e max 
df_reviews_cleaned1 = df_reviews_cleaned.withColumn('year', pyspark.sql.functions.split(df_reviews_cleaned['reviewTime'],', ').getItem(1))
df_reviews_cleaned2 = df_reviews_cleaned1.withColumn('month', pyspark.sql.functions.split(df_reviews_cleaned1['reviewTime'],' ').getItem(0))
df_reviews_cleaned2.dtypes

[('asin', 'string'),
 ('helpful', 'array<bigint>'),
 ('overall', 'double'),
 ('reviewText', 'string'),
 ('reviewTime', 'string'),
 ('reviewerID', 'string'),
 ('reviewerName', 'string'),
 ('summary', 'string'),
 ('unixReviewTime', 'bigint'),
 ('year', 'string'),
 ('month', 'string')]

In [26]:
#cast year to integer -- df_reviews1 and 2 are useful only to evaluate the year 
df_reviews_cleaned3 =df_reviews_cleaned2.withColumn('year_int', (df_reviews_cleaned2['year']).cast('integer'))
df_reviews_cleaned4 =df_reviews_cleaned3.withColumn('month_int', (df_reviews_cleaned3['month']).cast('integer'))

df_reviews_cleaned4.dtypes

[('asin', 'string'),
 ('helpful', 'array<bigint>'),
 ('overall', 'double'),
 ('reviewText', 'string'),
 ('reviewTime', 'string'),
 ('reviewerID', 'string'),
 ('reviewerName', 'string'),
 ('summary', 'string'),
 ('unixReviewTime', 'bigint'),
 ('year', 'string'),
 ('month', 'string'),
 ('year_int', 'int'),
 ('month_int', 'int')]

In [ ]:
type(df_reviews_cleaned4)

pyspark.sql.dataframe.DataFrame

In [ ]:
df_reviews_cleaned4.show()

+----------+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+----+-----+--------+---------+
|      asin|helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|year|month|year_int|month_int|
+----------+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+----+-----+--------+---------+
|0000031887| [0, 0]|    5.0|Perfect red tutu ...| 11 4, 2013|A2XVJBSRI3SWDI|             abigail|           Nice tutu|    1383523200|2013|   11|    2013|       11|
|0000031887| [1, 1]|    4.0|This was a really...|05 26, 2012|A2G0LNLN79Q6HR|       aj_18 "Aj_18"|Really Cute but r...|    1337990400|2012|   05|    2012|        5|
|0000031887| [1, 1]|    2.0|the tutu color wa...|02 17, 2013|A2R3K1KX09QBYP|      alert consumer|not very good mat...|    1361059200|2013|   02|    2013|        2|
|0000031887| [0,

In [27]:
df_reviews_cleaned4.createOrReplaceTempView('df_reviews_cleaned4')


In [ ]:
#take max and min year 

query6= """SELECT min(year_int), max(year_int)
           FROM df_reviews_cleaned4
        """

min_max_year= spark.sql(query6)
min_max_year.show()

+-------------+-------------+
|min(year_int)|max(year_int)|
+-------------+-------------+
|         1999|         2014|
+-------------+-------------+



In [ ]:
#take all unique values for years 
query7= """SELECT year_int
           FROM df_reviews_cleaned4
           GROUP BY year_int
           ORDER BY year_int DESC
        """

reviews_by_year= spark.sql(query7)
reviews_by_year.show(truncate=False)

+--------+
|year_int|
+--------+
|2014    |
|2013    |
|2012    |
|2011    |
|2010    |
|2009    |
|2008    |
|2007    |
|2006    |
|2005    |
|2004    |
|2003    |
|2002    |
|2001    |
|2000    |
|1999    |
+--------+



In [ ]:
#take all unique values for months 
query_by_months= """SELECT month_int
           FROM df_reviews_cleaned4
           GROUP BY month_int
           ORDER BY month_int DESC
        """

reviews_by_months= spark.sql(query_by_months)
reviews_by_months.show(truncate=False)

+---------+
|month_int|
+---------+
|12       |
|11       |
|10       |
|9        |
|8        |
|7        |
|6        |
|5        |
|4        |
|3        |
|2        |
|1        |
+---------+



In [28]:
#take reviews for 2014 - last year 

query8 = """SELECT *
           FROM df_reviews_cleaned4
           WHERE (year_int = 2014) 
         """
df_last_year= spark.sql(query8)
df_last_year.show()

+----------+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+----+-----+--------+---------+
|      asin|helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|year|month|year_int|month_int|
+----------+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+----+-----+--------+---------+
|0000031887| [0, 0]|    4.0|Bought it for my ...|01 23, 2014|A1P0IHU93EF9ZK|              Amanda|           i love it|    1390435200|2014|   01|    2014|        1|
|0000031887| [0, 0]|    5.0|Purchased it for ...|06 16, 2014|A1MCJONUQ78L9T|     Amazon Customer|My daughter loved...|    1402876800|2014|   06|    2014|        6|
|0000031887| [0, 0]|    5.0|Vey cute and perf...|02 18, 2014|A3CHBY0CB0O7PP|     Amazon Customer|                cute|    1392681600|2014|   02|    2014|        2|
|0000031887| [0,

In [ ]:
#analyse reviews for who give name and who don't give the name 
query9 = """SELECT *
           FROM df_reviews_cleaned2
           WHERE (year_int = 2014) AND (overall='5.0' or overall='4.0')
         """
df_last_year_same_overall = spark.sql(query9)
df_last_year_same_overall.show()

+----------+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+----+--------+
|      asin|helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|year|year_int|
+----------+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+----+--------+
|0000031887| [0, 0]|    4.0|Bought it for my ...|01 23, 2014|A1P0IHU93EF9ZK|              Amanda|           i love it|    1390435200|2014|    2014|
|0000031887| [0, 0]|    5.0|Purchased it for ...|06 16, 2014|A1MCJONUQ78L9T|     Amazon Customer|My daughter loved...|    1402876800|2014|    2014|
|0000031887| [0, 0]|    5.0|Vey cute and perf...|02 18, 2014|A3CHBY0CB0O7PP|     Amazon Customer|                cute|    1392681600|2014|    2014|
|0000031887| [0, 0]|    5.0|The tutu's was fo...|01 15, 2014| AM1CYMR007O6H|         anne foster|              T

In [ ]:
query10 = """SELECT *
           FROM df_reviews_cleaned2
           WHERE (year_int = 2014) AND (reviewerName='Amazon Customer')
         """
df_last_year_Amazon_Customer = spark.sql(query10)
df_last_year_Amazon_Customer.show()

+----------+-------+-------+--------------------+-----------+--------------+---------------+--------------------+--------------+----+--------+
|      asin|helpful|overall|          reviewText| reviewTime|    reviewerID|   reviewerName|             summary|unixReviewTime|year|year_int|
+----------+-------+-------+--------------------+-----------+--------------+---------------+--------------------+--------------+----+--------+
|0000031887| [0, 0]|    5.0|Purchased it for ...|06 16, 2014|A1MCJONUQ78L9T|Amazon Customer|My daughter loved...|    1402876800|2014|    2014|
|0000031887| [0, 0]|    5.0|Vey cute and perf...|02 18, 2014|A3CHBY0CB0O7PP|Amazon Customer|                cute|    1392681600|2014|    2014|
|0123456479| [0, 0]|    5.0|This was a gift f...|02 28, 2014|A16OS5CPJ2N2UA|Amazon Customer|Great little jewe...|    1393545600|2014|    2014|
|1617160040| [0, 0]|    4.0|I'm still in one ...| 04 7, 2014|A26P8S9G8NH4X7|Amazon Customer|         Works well.|    1396828800|2014|    2014|

In [ ]:
query11 = """SELECT *
           FROM df_reviews_cleaned2
           WHERE (year_int = 2014) AND (reviewerName <> 'Amazon Customer')
         """
df_last_year_not_Amazon_Customer = spark.sql(query11)
df_last_year_not_Amazon_Customer.show()

+----------+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+----+--------+
|      asin|helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|year|year_int|
+----------+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+----+--------+
|0000031887| [0, 0]|    4.0|Bought it for my ...|01 23, 2014|A1P0IHU93EF9ZK|              Amanda|           i love it|    1390435200|2014|    2014|
|0000031887| [0, 0]|    1.0|It was beautiful ...|04 17, 2014| AV056ETQ5RXLN|      anna alexander|               Cheap|    1397692800|2014|    2014|
|0000031887| [0, 0]|    5.0|The tutu's was fo...|01 15, 2014| AM1CYMR007O6H|         anne foster|              Tutus!|    1389744000|2014|    2014|
|0000031887| [0, 0]|    5.0|The tutu was the ...|05 10, 2014|A3QW1I2TL6WBS6|          Avidreader|         Perfec

I runned these queries to understand something about reviews and try to identify some column that have interesting insights. 

I decided to reduce the dataFrame considering only the last year (2014) because there are many variables, such as the price or the fashion trand, that could mismatch the reviews meaning for more than 1 year. 

In [ ]:
spark_shape(df_last_year)

(1916193, 13)

In [29]:
df_last_year= df_last_year.drop('year_int','month_int')
# better to do it later with merged dataframes df_rev_fin= df_last_months.dropDuplicates(['asin','reviewerName','unixReviewTime'])


In [ ]:
type(df_last_year)
df_last_year.dtypes

[('asin', 'string'),
 ('helpful', 'array<bigint>'),
 ('overall', 'double'),
 ('reviewText', 'string'),
 ('reviewTime', 'string'),
 ('reviewerID', 'string'),
 ('reviewerName', 'string'),
 ('summary', 'string'),
 ('unixReviewTime', 'bigint'),
 ('year', 'string'),
 ('month', 'string')]

Now I have to merge the two dataFrames and drop duplicates 

In [36]:
df_last_year.createOrReplaceTempView('df_last_year')
df_prd_cleaned5.createOrReplaceTempView('df_prd_cleaned5')

In [37]:
#merging 
from pyspark.sql.functions import collect_set
merged_data = spark.sql("""SELECT r.asin, r.helpful, r.overall, r.reviewText, r.reviewTime, r.reviewerID, r.reviewerName, r.summary, r.unixReviewTime, r.year, r.month, p.categories, p.price, p.title, p.brand
                       FROM df_last_year as r INNER JOIN df_prd_cleaned5 as p ON r.asin=p.asin 
                      """)

In [ ]:
type(merged_data)

pyspark.sql.dataframe.DataFrame

In [87]:
#check null values for merged_data dataFrame 
merged_data.select([count(when(col(c).isNull(), c)).alias(c) for c in merged_data.columns]).show()

+----+-------+-------+----------+----------+----------+------------+-------+--------------+----+-----+----------+-----+-----+-----+
|asin|helpful|overall|reviewText|reviewTime|reviewerID|reviewerName|summary|unixReviewTime|year|month|categories|price|title|brand|
+----+-------+-------+----------+----------+----------+------------+-------+--------------+----+-----+----------+-----+-----+-----+
|   0|      0|      0|         0|         0|         0|           0|      0|             0|   0|    0|         0|    0|    0|    0|
+----+-------+-------+----------+----------+----------+------------+-------+--------------+----+-----+----------+-----+-----+-----+



In [93]:
merged_data.createOrReplaceTempView('merged_data')

In [94]:
#chenck primary key: the combination asin-reviewText-reviewerName must have not duplicated values; so we have to check for duplicated values 
check_duplicated_value_for_PK=spark.sql("""SELECT COUNT(*)
                                           FROM merged_data
                                           GROUP BY asin, unixReviewTime, reviewerName
                                           Having COUNT(*)> 1""")
check_duplicated_value_for_PK.show()
spark_shape(check_duplicated_value_for_PK)

+--------+
|count(1)|
+--------+
|       2|
|       2|
|       2|
|       2|
|       2|
|       2|
|       2|
|       2|
|       2|
|       2|
|       2|
|       2|
|       2|
|       2|
+--------+



(14, 1)

In [89]:
#drop duplicates with respect to primary key --- probably duplicates because of inner join 
merged_data_cleaned = merged_data.dropDuplicates(['asin','reviewerName','unixReviewTime'])
#m2= merged_data_cleaned.drop('summary','categories','price')


In [ ]:
spark_shape(merged_data_cleaned)

(293701, 15)

The shape of the merged dataFrame is smaller enough to analyse data in Pandas, so I have to convert sparkDataFrame into PandasDataFrame

In [39]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
df_reviews_products= merged_data_cleaned.select("*").toPandas()

In [40]:
df_reviews_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293701 entries, 0 to 293700
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   asin            293701 non-null  object 
 1   helpful         293701 non-null  object 
 2   overall         293701 non-null  float64
 3   reviewText      293701 non-null  object 
 4   reviewTime      293701 non-null  object 
 5   reviewerID      293701 non-null  object 
 6   reviewerName    293701 non-null  object 
 7   summary         293701 non-null  object 
 8   unixReviewTime  293701 non-null  int64  
 9   year            293701 non-null  object 
 10  month           293701 non-null  object 
 11  categories      293701 non-null  object 
 12  price           293701 non-null  float64
 13  title           293701 non-null  object 
 14  brand           293701 non-null  object 
dtypes: float64(2), int64(1), object(12)
memory usage: 33.6+ MB


In [ ]:
df_reviews_products.shape

(293701, 15)

**Aspect Extraction, Word Embedding and Aspect Classification**

Following steps: 

-clean reviewText variables 

-run GloVe 

-run the BiGRU-CRF models, the LSTM-CRF model and the CRF model 


In [ ]:
print(df_reviews_products.head())
print(df_reviews_products.columns)
print(df_reviews_products.shape)

         asin  ...               brand
0  1848572972  ...  Little Tiger Press
1  8921463267  ...         Hello Kitty
2  B000034DLG  ...               elope
3  B000034DLG  ...               elope
4  B00004U1J2  ...              Disney

[5 rows x 15 columns]
Index(['asin', 'helpful', 'overall', 'reviewText', 'reviewTime', 'reviewerID',
       'reviewerName', 'summary', 'unixReviewTime', 'year', 'month',
       'categories', 'price', 'title', 'brand'],
      dtype='object')
(293701, 15)


In [98]:
#check rating values and balancing 
def rating_sentiment(value):
  if (int(value) == 1 or int(value) == 2):
    return "Negative"
  if (int(value) == 3): 
    return "Neutral"
  else:
    return "Positive"

sentiment_overall  = df_reviews_products.overall.apply(rating_sentiment)
sentiment_overall.value_counts()


Positive    220172
Negative    44623 
Neutral     28906 
Name: overall, dtype: int64

In [ ]:
#since the dataset is not balanced, I'll randomly choose same number of positive and negative overall values (and as a consequence rows associated)

In 2014 ratingd between 4 and 5 are much more, so I expect that reviews are positive. I think that I have to balance the number of positive and negative reviews, because a dataset so unbalanced can cause issues. But I see before the results. 

We will remove stop words from all the reviews. 

In [42]:
#lower case for reviews  if it is not a stopword 
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop = set(stopwords.words("english"))


def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]

    return " ".join(text)



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [43]:
df_reviews_products['reviewText'] = df_reviews_products['reviewText'].map(remove_stopwords)

In [ ]:
df_reviews_products.reviewText

0         short book that's great infant young toddler. ...
1         shoes extremely pretty. pink like pictured, sa...
2         bought home preschool well pleased.they well m...
3         son dress school insect day! absolutely adorab...
4         spots elastic joined soft vinyl tore days wear...
                                ...                        
293696    first all, watch pictured dz1657, dz4323, surp...
293697         perfect durable. bought take vacation beach!
293698                                 ok quality expected.
293699    runs small, definitely shorter shows on-line. ...
293700                              great looking backpack!
Name: reviewText, Length: 293701, dtype: object

explaination of the code: https://towardsdatascience.com/sentiment-analysis-using-lstm-and-glove-embeddings-99223a87fe8e







Remove tags and punctuation 




In [44]:
def remove_tags(string):
    result = re.sub('<.*?>','',string)
    return result
    
df_reviews_products['reviewText']= df_reviews_products['reviewText'].apply(lambda cw : remove_tags(cw))
df_reviews_products['reviewText']= df_reviews_products['reviewText'].str.replace('[{}]'.format(string.punctuation), ' ')

In [45]:
df_reviews_products['overall']= df_reviews_products['overall'].astype(int)

In [96]:
reviews = df_reviews_products['reviewText']
reviews

0         short book that s great infant young toddler  pictures catch attention little one  naturally  love makes sounds  imagine daughter s dismay stopped making kissing sounds day got it  no  one kids sits incessantly pushes buttons batteries run immediately  got new batteries replace  takes 3 small calculator batteries   hoping problem batteries old replaced three  guess what  worked 10 button presses stopped again  since large pack batteries  one replacement daughter begged months later  thing   a handful button pushes nothing  daughter wants throw book trash  hardly blame her                                                                                                                                                                                                                                                                                                                                                                                                                            

Create sequences and label

In [97]:
#review list and sentiment are useful to then compute training and test set 
#create a list for reviews and sentiment about reviews -- the list could be useful to make descriptive analysis later on 
for i in range(len(reviews)):
  reviews_list.append(reviews[i])

sentiment = df_reviews_products['overall']

sentiment.value_counts()

5    166180
4    53992 
3    28906 
1    26639 
2    17984 
Name: overall, dtype: int64

In [ ]:
reviews_list[0]

'short book that s great infant young toddler  pictures catch attention little one  naturally  love makes sounds  imagine daughter s dismay stopped making kissing sounds day got it  no  one kids sits incessantly pushes buttons batteries run immediately  got new batteries replace  takes 3 small calculator batteries   hoping problem batteries old replaced three  guess what  worked 10 button presses stopped again  since large pack batteries  one replacement daughter begged months later  thing   a handful button pushes nothing  daughter wants throw book trash  hardly blame her '

Counting words contained in the reviews 

In [99]:
#count unique words 
from collections import Counter

# Count unique words
def counter_word(text):
    count = Counter()
    for i in text.values:
        for word in i.split():
            count[word] += 1
    return count

In [100]:
#unique words count (non total words count)
text= reviews
counter = counter_word(text)
len(counter)

78151

In [101]:
#maybe try to put count of words in ascending order
type(counter)
counter

Counter({'short': 5156,
         'book': 1153,
         'that': 12126,
         's': 32570,
         'great': 74336,
         'infant': 187,
         'young': 1087,
         'toddler': 582,
         'pictures': 2338,
         'catch': 552,
         'attention': 1338,
         'little': 32441,
         'one': 62637,
         'naturally': 148,
         'love': 52965,
         'makes': 7874,
         'sounds': 351,
         'imagine': 779,
         'daughter': 10621,
         'dismay': 32,
         'stopped': 1524,
         'making': 2486,
         'kissing': 4,
         'day': 18200,
         'got': 26256,
         'it': 76081,
         'no': 1691,
         'kids': 3416,
         'sits': 649,
         'incessantly': 8,
         'pushes': 85,
         'buttons': 1691,
         'batteries': 1728,
         'run': 3234,
         'immediately': 1745,
         'new': 11372,
         'replace': 3593,
         'takes': 1970,
         '3': 11437,
         'small': 28589,
         'calculator': 20

 **Part of Speech tagging**

associate to aspect BIO Classification --- work in progress

Build Training and Test set to extract aspect and predict sentiment classification 

In [ ]:
#compute label and convert to one-hot format 
y = np.array(list(map(lambda x: 1 if x > 3 else 0, sentiment)))
y

array([0, 0, 1, ..., 0, 1, 1])

In [ ]:
#X_train, X_test,Y_train, Y_test = train_test_split(df_reviews_products.reviewText, y, test_size=0.2, random_state = 45)

In [ ]:
train_size = int(df_reviews_products.shape[0] * 0.8)

train_sentences = df_reviews_products.reviewText[:train_size]
train_labels = df_reviews_products.overall[:train_size]

test_sentences = df_reviews_products.reviewText[train_size:]
test_labels = df_reviews_products.overall[train_size:]

In [ ]:
len(train_sentences)

234960

In [ ]:
#len(Y_train), len(X_train), len(Y_test), len(X_test)

(234960, 234960, 58741, 58741)

In [ ]:
#maybe I can do a prediction on it


Tokanization - only consider the X set; because we consider only reviews for this part 

In [ ]:
len(counter)

78151

In [ ]:
#train_sequences = X_train
#train_labels = Y_train

#test_sequences = X_test
#test_labels = Y_test



In [103]:
num_words = len(counter)
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(train_sentences)

In [104]:
# Max number of words in a sequence(review texts in our case)
max_length = 150

In [105]:
words_to_index = tokenizer.word_index
len(words_to_index)

69169

The number of different words in the dataset after the
pre-processing is counted, the number of occurrences of each
word, the length of the largest word included in each review,
and the length of the word contained in each review in the
calculated dataset. The average review length is used as the
ﬁxed length of words in each review. If the review length is
larger than the ﬁxed length, it is intercepted, and if the review
length is smaller than the ﬁxed length, 0 will be added

In [106]:
#now we give an index to each word 

words_to_index

{'watch': 1,
 'it': 2,
 'great': 3,
 'like': 4,
 'i': 5,
 'one': 6,
 'good': 7,
 'would': 8,
 'love': 9,
 'well': 10,
 'time': 11,
 'nice': 12,
 'fit': 13,
 'really': 14,
 '34': 15,
 'quality': 16,
 'price': 17,
 'size': 18,
 'bought': 19,
 'bag': 20,
 'wear': 21,
 'get': 22,
 's': 23,
 'looks': 24,
 'little': 25,
 'small': 26,
 'look': 27,
 'perfect': 28,
 'use': 29,
 'made': 30,
 'got': 31,
 'much': 32,
 'm': 33,
 'also': 34,
 'buy': 35,
 'product': 36,
 'big': 37,
 'looking': 38,
 'recommend': 39,
 'even': 40,
 'color': 41,
 'band': 42,
 'still': 43,
 'back': 44,
 'easy': 45,
 'day': 46,
 'work': 47,
 'fits': 48,
 'them': 49,
 'first': 50,
 '5': 51,
 'comfortable': 52,
 'cute': 53,
 'bit': 54,
 'better': 55,
 'pretty': 56,
 'gift': 57,
 '2': 58,
 'long': 59,
 'watches': 60,
 've': 61,
 'could': 62,
 'hat': 63,
 'way': 64,
 'large': 65,
 'ordered': 66,
 'loves': 67,
 'cheap': 68,
 'face': 69,
 'two': 70,
 'see': 71,
 'enough': 72,
 'need': 73,
 'happy': 74,
 'put': 75,
 'old': 76,
 '

In [107]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)

In [108]:
train_sequences[0]

[290,
 982,
 92,
 23,
 3,
 3862,
 1068,
 1692,
 581,
 1711,
 932,
 25,
 6,
 3806,
 9,
 175,
 2255,
 1343,
 119,
 23,
 8476,
 822,
 548,
 34022,
 2255,
 46,
 31,
 2,
 762,
 6,
 406,
 1568,
 14671,
 4868,
 771,
 754,
 425,
 755,
 31,
 107,
 754,
 386,
 698,
 109,
 26,
 2909,
 754,
 668,
 190,
 754,
 76,
 730,
 331,
 390,
 929,
 276,
 243,
 546,
 7080,
 822,
 181,
 125,
 65,
 408,
 754,
 6,
 430,
 119,
 10390,
 220,
 652,
 114,
 593,
 5038,
 546,
 4868,
 295,
 119,
 876,
 1120,
 982,
 1871,
 1505,
 3876,
 737]

In [109]:
train_sentences[0]

'short book that s great infant young toddler  pictures catch attention little one  naturally  love makes sounds  imagine daughter s dismay stopped making kissing sounds day got it  no  one kids sits incessantly pushes buttons batteries run immediately  got new batteries replace  takes 3 small calculator batteries   hoping problem batteries old replaced three  guess what  worked 10 button presses stopped again  since large pack batteries  one replacement daughter begged months later  thing   a handful button pushes nothing  daughter wants throw book trash  hardly blame her '

Padding 

In [110]:
#pad sequences --- replace sentences with word vectors
from keras.preprocessing.sequence import pad_sequences

train_padded = pad_sequences(
    train_sequences, maxlen=max_length, padding="post", truncating="post"
)

In [111]:
#I guess this is the vector for the first sentence ---- zeros because the sentence at position zero is shorter then the largest sentence in the text 
train_padded[0]

array([  290,   982,    92,    23,     3,  3862,  1068,  1692,   581,
        1711,   932,    25,     6,  3806,     9,   175,  2255,  1343,
         119,    23,  8476,   822,   548, 34022,  2255,    46,    31,
           2,   762,     6,   406,  1568, 14671,  4868,   771,   754,
         425,   755,    31,   107,   754,   386,   698,   109,    26,
        2909,   754,   668,   190,   754,    76,   730,   331,   390,
         929,   276,   243,   546,  7080,   822,   181,   125,    65,
         408,   754,     6,   430,   119, 10390,   220,   652,   114,
         593,  5038,   546,  4868,   295,   119,   876,  1120,   982,
        1871,  1505,  3876,   737,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [112]:
#vector for the second sentence 
train_padded[1]

array([ 173,  388,   56,  478,    4,  547, 9374, 1110,   38,    3,   78,
         30,    7,  632,  104,  190, 1249,  375,  997, 1150,   21,  191,
       3598, 3146,  268,   85,  644,  651, 3146, 1575, 2058,   56,  188,
        246,   30,    3,   16,  632,  204,    8,   95, 1249,  134,   64,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0], d

In [113]:
# Pad the sentences for testset 

test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(
    test_sequences, maxlen=max_length, padding="post", truncating="post"
)


In [114]:
#compare phrase with vector representing phrase 
print(df_reviews_products.iloc[0].reviewText)
print(train_sequences[0])



short book that s great infant young toddler  pictures catch attention little one  naturally  love makes sounds  imagine daughter s dismay stopped making kissing sounds day got it  no  one kids sits incessantly pushes buttons batteries run immediately  got new batteries replace  takes 3 small calculator batteries   hoping problem batteries old replaced three  guess what  worked 10 button presses stopped again  since large pack batteries  one replacement daughter begged months later  thing   a handful button pushes nothing  daughter wants throw book trash  hardly blame her 
[290, 982, 92, 23, 3, 3862, 1068, 1692, 581, 1711, 932, 25, 6, 3806, 9, 175, 2255, 1343, 119, 23, 8476, 822, 548, 34022, 2255, 46, 31, 2, 762, 6, 406, 1568, 14671, 4868, 771, 754, 425, 755, 31, 107, 754, 386, 698, 109, 26, 2909, 754, 668, 190, 754, 76, 730, 331, 390, 929, 276, 243, 546, 7080, 822, 181, 125, 65, 408, 754, 6, 430, 119, 10390, 220, 652, 114, 593, 5038, 546, 4868, 295, 119, 876, 1120, 982, 1871, 1505, 38

In [115]:
#check inverse --- word to index derives from sentences tokanized 
reverse_words_to_index = dict([(value, key) for (key, value) in words_to_index.items()])

In [116]:
def decode(text):
    return " ".join([reverse_words_to_index.get(i, "?") for i in text])

In [117]:
decode(train_sequences[0])

'short book that s great infant young toddler pictures catch attention little one naturally love makes sounds imagine daughter s dismay stopped making kissing sounds day got it no one kids sits incessantly pushes buttons batteries run immediately got new batteries replace takes 3 small calculator batteries hoping problem batteries old replaced three guess what worked 10 button presses stopped again since large pack batteries one replacement daughter begged months later thing a handful button pushes nothing daughter wants throw book trash hardly blame her'

In [121]:
decode(test_sequences[0])

'plastic cheap ugly uncomfortable wear piece junk china don t waste money watch get another brand'

I have tokenized reviews in text and then I inserted each token in array representing tokens. The maximum length of each array is given by the review that has maximum length. I had to uniformate the array lengths because for NN we need to have array of the same length 

**The last part is basically related to word embedding, feature extraction and sentiment classification**


Now, in this second part, the focus is on reviews and analysis of test contained in each review.

a) GloVe Model

b) BiGru Model

c) CFR without BiGRU Model and with BiGRU Model 

d) comparison between BiGRU Model and LSTM-Model 

**Glove to build a word embedding vector**

Now we have to convert words (tokenized text) in word vector and, in order to do this, we use GloVe that is a word embedding model.

The aim of gloVe is to build word vector and represent similar words in the same vector space. 



In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2021-10-23 17:29:06--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-10-23 17:29:06--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-10-23 17:29:06--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
!unzip glove*.zip
!ls
!pwd

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       
derby.log  glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip  sample_data
drive	   glove.6B.200d.txt  glove.6B.50d.txt	 metastore_db
/content


In [ ]:
def read_glove_vector(glove_vec):
  with open(glove_vec, 'r', encoding='UTF-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)



  return word_to_vec_map

In [ ]:
word_to_vec_map = read_glove_vector('/content/drive/MyDrive/glove.6B.100d.txt')

In [ ]:
#add +1 because the matrix starts from position 0 and the length from position 1
vocab_len = len(words_to_index) + 1
vocab_len

69170

In [ ]:
embed_vector_len = word_to_vec_map['moon'].shape[0]
embed_vector_len

100

In [ ]:
emb_matrix = np.zeros((vocab_len, embed_vector_len))
emb_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
# Max number of words in a sequence
max_length = 150

In [ ]:
for word, index in words_to_index.items():
  embedding_vector = word_to_vec_map.get(word)
  if embedding_vector is not None:
    emb_matrix[index, :] = embedding_vector

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=max_length, weights = [emb_matrix], trainable=False)

In [ ]:
#The main function of the embedded layer is to represent the text statement as a weighted word vector matrix; all the sententes are represented by the embedded matrix as a collection of embedded vectors
embedding_layer

In [ ]:
#before embedded matrix with zeros and then insert embedded vector inside 
emb_matrix


array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [-0.38264 , -0.089682,  0.024723, ...,  0.16366 ,  0.43887 ,
         0.28925 ],
       [-0.30664 ,  0.16821 ,  0.98511 , ..., -0.38775 ,  0.36916 ,
         0.54521 ],
       ...,
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.92085 ,  0.30269 ,  0.35859 , ...,  0.35185 , -0.2279  ,
        -0.73066 ],
       [-1.3046  ,  0.055802,  0.25823 , ..., -0.71069 , -0.35377 ,
        -0.63405 ]])

In [ ]:
embedding_vector

array([-1.3046e+00,  5.5802e-02,  2.5823e-01,  3.1793e-01, -1.6898e-01,
       -1.1317e+00,  6.1549e-01, -1.0256e+00, -7.3427e-02, -4.9743e-01,
       -6.4630e-01,  1.2031e-01, -1.4924e-01, -4.2913e-01, -6.1349e-01,
       -2.6137e-01, -1.7414e-01,  6.4564e-01,  7.2441e-01,  1.9579e-01,
        3.5705e-01,  5.7211e-01, -1.1064e-01,  4.9322e-01,  5.2623e-01,
        3.0543e-02, -8.4493e-01, -5.2750e-01, -4.5739e-01,  2.3514e-01,
        1.1006e+00, -5.6729e-01,  3.5634e-01, -1.1462e-01,  2.3368e-01,
       -8.4738e-01,  5.5798e-02,  2.6828e-01, -6.2954e-02,  5.6889e-01,
        1.3936e-01, -4.4661e-01, -2.2147e-01, -6.7564e-02,  1.6134e-01,
        1.5776e-03, -6.8686e-01,  5.2118e-02,  1.1329e+00,  3.5537e-01,
       -7.3704e-01,  1.9332e-02, -7.0289e-01, -6.2026e-02, -8.0289e-01,
        1.4799e+00,  1.1485e+00,  9.4612e-01,  3.1200e-01, -7.0017e-01,
        3.4272e-02,  9.3862e-02,  4.8533e-01, -1.0437e+00, -2.0160e-01,
        2.9753e-01,  2.4468e-01,  3.5931e-01, -2.0053e+00,  3.13

**Prossimi step**

-gru layer per aspect extraction 

-CRF 

Part of speech per la predizione 